## Project 2 - Advanced Lane Finding

### You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

NOTES ON CHANGES FOR 2ND SUBMISSION CAN BE FOUND AT THE END OF THIS DOCUMENT.

---

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./camera_cal/calibration1.jpg "Input"
[image2]: ./output_images/undistorted_checkerboard.jpg "Undistorted"
[image3]: ./test_images/test1.jpg "Test Image"
[image4]: ./output_images/undistorted_test1.jpg "Undistorted Test Image"
[image5]: ./output_images/binary_test1.jpg "Binary Thresholded Image"
[image6]: ./output_images/warped_test1.jpg "Warped Image - Top Perspective"
[image7]: ./output_images/lanes_test1.jpg "Lanes"
[image8]: ./output_images/roadwarped_test1.jpg "Original Image with Lanes Marked"
[image9]: ./output_images/tracked_test1.jpg "Final Annotated Images"
[video1]: ./project_video.mp4 "Video"

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points

### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---

### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  [Here](https://github.com/udacity/CarND-Advanced-Lane-Lines/blob/master/writeup_template.md) is a template writeup for this project you can use as a guide and a starting point.  

This is it.

### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is contained in the first code cell of the IPython notebook located in "./Advanced Lane Finding.ipynb".

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  The distortion coefficients were stored in "calibration_pickle.p" so they could be used in later portions of the code.  The 

In the second code cell in the notebook, I test the calibration.  First, I read the distortion coefficients from the pickle file. 
Then I test using one of the checkerboard sample files (calibration1.jpg).  Here are the results:
Original file:
![Input][image1]

Undistorted file:
![Undistorted][image2]

### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

In the second code cell, I also apply the same undistortion correction to one of the sample input files.
Input file:
![Input][image3]

Output:
![Output][image4]

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

The third code cell contains a group of test code I used to eventually build the pipeline for processing videos.  While building this code cell, images were save at every step to better understand what processing each step was performing and how parameter adjustments affected the output.  All output files can be found in the folder "./output_images/".

Functions that we created as part of the quizzes are contained in the third code cell including sobel thresholding, magnitude thresholding, direction thresholding, color thresholding and window masking.

The code reads each test image from "./test_images", applies the distortion coefficients and writes the file with the prefix "undistorted_".  Here's an example:
![Undistorted "test1"][image4]

I applied a number of the thresholding techniques to each image (sorbel & color, primarily) and and created combined binary threshold images.  These files were written with the prefix "binary_".  Here's an example:
![Binary "test1"][image5]

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

I defined a trapezoid source shape and a rectangular destination shape and used the cv2 functions getPerspectiveTransform and warpPerspective Transform to perform the necessary transform and generate a top perspective of the lanes.  These images were written with the prefix "warped_".  Here's an example:
![Top Perspective (warped)][image6]

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

The sliding windows technique was used to identify lane-line pixels.  The "tracker" class in the third code cell was used to find the window centroids.  This allowed the lane-lines to be detected and highlighted.  The lanes were marked on the warped images and saved with prefix "lanes_".  Here's an example:
![Lanes][image7]

Lane lines polygons were added to the images (red for left and blue for right) with a middle marker (in green) that defined the area between the lane lines.  The images were unwarped using cv2's warpPerspective with the inverse transformation matrix to convert it back to a normal view (from the car).  Some color enhancements were made to the image to emphasize the lane lines.  These images were saved with the prefix "roadwarped_".  Heres' and example:
![Road with lanes marked][image8]

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

The variable curverad is calculated using np.polyfit and the standard curvature calcuation:
curverad = ((1 + (2*curve_fit_cr[0]*yvals[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5) /np.absolute(2*curve_fit_cr[0])

The center of the camera's perspective was used as an indication of the center of the car.  This was used to calculate left or right of center.

The radius of curvature and center position of the car were added to the image as annotation using cv2.putText.  Here's an example of a final annotated image:
![Final Annotated Road][image9]

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

This is the "roadwarped_" image presented earlier.  Here it is again:
![Road with lanes marked][image8]

---

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

I copied the code from the third code cell into the fourth code cell, creating the function "process_image" by removing the code that read individual image files along with intermediate image storage to the file system and returning the final image.  Using VideoFileClip from moviepy.editor, I read the video file and passed it through the processing, saving it to file "output1_tracked.mp4".

Here's a link to the processed video - [link to my video result](./output1_tracked.mp4)

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

While coding the project, I found myself referring back to notes and code examples I had collected while working through the "class assignments" and quizzes.  I believe that for the next project, I'll be more diligent in this area.  I found that I didn't fully understand several concepts (like thresholding and creating the source shape for transforms).  This required additional time while completing the projecting.

I also noticed that I included virtually no error handling.  My lack of Python knowledge is really hindering me in this area.  As I gain more experience, I'd expect that more error trapping will become common place in my code.

### 2nd Submission

#### 1. General notes

All changes related to this 2nd submission are flagged in the code with a "2nd submission" comment.

I eliminated the "tracker" class definition from the previous submission.  This became the funcition "find_window_centroids" that is located in the third code cell.  I created "process_image" in the third code cell for the pipeline function.  I also defined a local variable outside of "process_image" for "recent_centers" that "process_image" uses to store centers for averaging.  "process_image" has the option to write the intermediate files that are generated (defaulted to False) so the same function can be used for both individual image processing and called for the video frame processing (no file saves).

#### 2. Adjusted lane line perspective transform

To eliminate misidentifying cars as lane markers and make lane searching easier, I changed the offset percentage in the destination for the perspective transform from 0.25 to 0.15. This makes the lanes in the transformed "bird's-eye" view fill up more of the frame.

The change can be found in the "process_image" function in the third code cell.
`offset = img_size[0] * .15`

#### 3. Adjusted the "meters per pixel" scaling

I updated the scaling factors for x and y "meters per pixel" based on suggestions from the reviewer.  This corrected the calculations for both radius and car position.

The change can be found in the "process_image" function in the third code cell.
```xm_per_pix = 3.7/500
ym_per_pix = 30/720
```

#### 4. Corrected the car position indicator

My logic for left and right detection were reversed.  I corrected this logic.  I didn't find any lateral shift caused by the perspective transform and the calculation for the car position results were found to be near the expected 0.5m "left of center".

The change can be found in the "process_image" function in the third code cell.
```side_pos = 'left'
if center_diff <= 0:
   side_pos = 'right'
```

#### 5. Continued Experimentation

The pipeline generally works well on the sample images as shown in the images above.  I performed some additional code enhancements to improve the processing.  I added a red channel theshold to the function "color_threshold" but found that it caused more problems that in solved.  I adjusted the threshold settings and the averaging and smoothing functions of "find_window_centroids" which improved the lane marker "jitter" in the video.  I'll continue to experiment to improve the pipeline.